# <center> Recommender Systems </center>
## <center> Content-based & Hybrid approaches. Week 3</center>


* Hybrid approaches classification
* Lightfm paper & library
* Case study: next basket recommendations (X5 retail hero)

Collaborative filtering vs. Content models

В контентных моделях есть недостаток, что нет так называемого трансфер лернинга. То есть, предсказание по пользователя делается отдельно по его фичам (изолированно от других пользователей). Поэтому нужно довольно много хороших признаков по клиентам, что тоже не дает в полной мере решить проблему холодного старта. 



### LightFM. 
#### Original paper: Metadata Embeddings for User and Item Cold-start Recommendations (2015). 

* Это гибридный подход коллаборативной фильтрации и контентной модели, которая предсталвяет эмбеддинги пользователей и эмбеддинги объектов как линейные комбинации из обученных векторов известных признаков - т.е. суммы новых латентных признаков. При этом, это позволяет обучать модель как в режиме без признаков, так и с ними, решая проблему холодного старта (т.к. по новым пользователям и объектам можно использовать их признаки) и проблему слишком разреженных данных (high sparsity problem). LightFM умеет хорошо работает как с плотными, так и с разреженными данными. Как бонус, эмбеддинги признаков кодируют в себе семантическую информацию по аналогии с подходами для получения эмбеддингов слов (например, w2v).

*  Формализация. Пусть: <br> $U$ - множество пользователей, <br> $I$ - множество объектов, <br> $F^{U}$ - множество признаков пользователей, <br> $F^{I}$- множество признаков объектов. <br>
Все пары $(u, i) \in U × I$ - это объединение всех положительных $S^{+}$ и отрицательных $S^{-}$ интеракций. 

Каждый пользователь описан набором заранее известных признаков (мета данных) $f_u \subset F^U$, то же самое для объектов  $f_i \subset F^I$.

Латентное представление пользователя представлено суммой его латентных векторов признаков: 

$$q_u = \sum_{j \in f_u} e^U_j$$

Аналогично для объектов: $$p_u = \sum_{j \in f_i} e^I_j$$

Так же, по пользователю и объекту есть смещения (bias): 

$$b_u = \sum_{j \in f_u} b^U_j$$

$$b_i = \sum_{j \in f_i} b^I_j$$

Предсказание из модели будет получать через скалярное произведение эмбедингов пользователя и объекта. 

$$\hat r_{ui} = f (q_u \cdot p_i + b_u + b_i)$$

Функция f() может быть разной, автор статьи выбрал сигмоиду, поскольку использовал бинарные данные.

$$f(x) = \frac{1}{1 + exp(-x)}$$

Задача оптимизации будет сформулирована как максимизация правдоподобия (данных при параметрах). Обучать модель будет с помощью стохастического градиентного спуска (AdaGrad).

$$L(e^U, e^I, b^U, b^I) = \prod_{(u, i) \in S^+} \hat r_{ui} \cdot  \prod_{(u, i) \in S^-} (1 - \hat r_{ui})$$


Если признаков нет, то это равноценно тому, чтобы подать на вход единичную матрицу, и просто учить эмбеддинги пользователей и объектов - сводим задачу к коллаборативной фильтрации. 
Но тогда у нас проблема холодного старта - скрытые векторы для индикаторных переменных (единиц на главной диагонали) не могут быть оценены для новых пользователей или товаров. 

Если наоборот, есть только фичи, а индикаторных переменных нет, то модель все равно не сводится к контентной, так как LightFM оценивает эмбеддинги признаков через факторизацию матрицы взаимодействий пользователей. В контентных моделях, в которых используется снижение размерности, факторизация происходит по матрицам совстречаемости контента (объектов).

### Гибридные системы

Одна из классификаций:

<img src='https://raw.githubusercontent.com/anamarina/RecSys_course/main/week3/images/taxonomy.png'>


Команда Bellkor Pragmatix Chaos выиграла премию в 1 миллион долларов за решение, в котором комбинировалось 107 различных алгоритмов; их программа повысила точность рекомендательного движка Cinematch на Netflix на 10.06%. Начиная с Netflix Prize 2006-2009, более активно началось развитие гибридных рекомендательных систем. 

## Пример

Кстати, к какому типу из классификации гибридных алгоритмов он относится?

<img src='https://raw.githubusercontent.com/anamarina/RecSys_course/main/week3/images/twolevel.png'>

- Какие здесь могут быть плюсы/минусы при использовании подхода?


**Плюсы:**
- На первом уровне могут быть модели, в которые нельзявс вставить признаки, и при сборе выборке из предиктов разных моделей мы можем их добавить.
- Все преимещуства самого бустинга, включая то, что добиваемся более высокого качества, когда учимся на ошибках предыдущих алгоритмов.
- Довольно удобно считать feature importance и смотреть, какие модели вносят больший вклад в предсказания, а так же проанализировать значимость дополнительных признаков при ранжировании.
- На первом уровне можно взять простые модели, которые будут довольно грубо отсекать большинство плохих кандидатов в рекомендации, но зато делать это быстро и сужая пространство хороших кандидатов. 
- Есть возможность сделать быстрый для прода пайплайн, особенно учитывая, что бейзлайны на статистиках считаются очень быстро. Например, для ALS и xgboost есть имплементации на Pyspark. 

**Минусы:**
- Если на первом уровне сильная end-to-end модель и незначимые фичи на втором уровне, прироста после бустинга не будет. 
- Выборка для бустинга состоит из предсказаний после первого уровня, то есть, есть зависимость в последовательности получения данных, поэтому если обновляются кандидаты на первом уровне, часто бустинг тоже приходится обучать с нуля (т.к. предсказания с первого уровня могли сильно поменяться). 
- Появляются дополнительные параметры для тюнинга - например, число моделей на первом уровне, параметр k - длина списка рекомендаций из каждой модели первого уровня. 
- Обычно нужен дополнительный анализ предиктов с первой модели, потому что если предикты из моделей сильно скоррелированны, то будет линейная зависимость в данных. 

И так далее. 


#### Case study: next purchase recommendation using cascade (two-level) approach. 

Рассмотрим задачу построения рекомендательной системы для задачи рекомендации следующей покупки на соревновании [X5 retail hero](https://ods.ai/competitions/x5-retailhero-recommender-system/data). 

По каждому пользователю должны выдать топ-20 рекомендованных товаров и посчитать метрику качества MNAP@20.

<img src='https://raw.githubusercontent.com/anamarina/RecSys_course/main/week3/images/mnap.png' width=500 height=300>

где $r_u(i)$ — бинарная оценка за тестовый период купил ли клиент товар или нет, 

$n_u$ — количество фактически купленных товаров пользователем за тестовый период,

$U$ — множество пользователей.

In [6]:
!pip install lightgbm

In [7]:
import os

DATA_PATH = "./"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import warnings

warnings.filterwarnings("ignore")

import itertools
import pickle
import random
from datetime import datetime
from functools import partial

import lightgbm
import matplotlib.pyplot as plt
%matplotlib notebook
import numpy as np
import pandas as pd
import plotly.express as px
import scipy
import seaborn as sns
from lightfm import LightFM
from scipy.sparse import coo_matrix, csr_matrix, save_npz
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

OSError: dlopen(/Users/m.ananyeva/opt/anaconda3/lib/python3.8/site-packages/lightgbm/lib_lightgbm.so, 6): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /Users/m.ananyeva/opt/anaconda3/lib/python3.8/site-packages/lightgbm/lib_lightgbm.so
  Reason: image not found

In [ ]:
receipts = pd.read_csv('purchases_sample.csv') # выборка из датасета x5
receipts.head()

#### Preprocessing

- Фильтрация недостаточно активных пользователей
- Уменьшение размера множества объектов для рекомендаций (удалить непопулярные)



In [ ]:
ditrib = receipts['item'].value_counts()[:100].rename_axis('item').reset_index(name='counts')
fig = px.bar(ditrib, x="item", y="counts", color="counts")
fig.show()

In [ ]:
def filter_by_counts(df, min_item_cnt=10, min_items_per_user=0):
    '''
    min_item_cnt - minimum frequency of item's purchases among all receipts.
    min_items_per_user - minimum unique items per user.
    '''
    item_counts = df.item.value_counts()
    item_freq = set(item_counts[item_counts >= int(min_item_cnt)].index)
    df = df[df.item.isin(item_freq)]
    active_users = dict(receipts.groupby('party_rk')['item'].nunique() > min_items_per_user)
    idxs = {k for k, v in active_users.items() if v == True}
    df = df[df.party_rk.isin(idxs)]
    return df

In [ ]:
receipts = filter_by_counts(receipts, min_item_cnt=10, min_items_per_user=0)

#### Train-test split & cross-validation

- Сплит по пользователям
- Сплит по времени (обычный без нарушения хронологии, кросс-валидация с кумулятивным сплитом, скользящее временное окно)
- Сплит с маскировкой n% интеракций и в train, и в test. 

1) 20% of all interaction pairs
are randomly assigned to the test set, but all items and
users are represented in the training set

2) item cold-start scenario: all interactions pertaining to 20%
of items are removed from the training set and added to
the test set. T

1) Как делить данные на train и test?


<img src='https://raw.githubusercontent.com/anamarina/RecSys_course/main/week3/images/spliting.png' width=500 height=300>


In [ ]:
def split_by_time(df, split_date="2019-02-18", filter_train=10, filter_test=5):
    '''
    Function splits a data set into train and test by time.

    split_date - date for split 75% train and 25% test.
    filter_train - threshold for minimum number of unique items purchased in train period.
    filter_test - threshold for minimum number of unique items purchased in test period.
    '''
    df.ymd = df.ymd.apply(lambda x: x[:10])
    receipts_train = df[df.ymd < split_date]
    receipts_test = df[df.ymd >= split_date]
    
    user_train_cnt = receipts_train[["party_rk", "item"]].drop_duplicates().party_rk.value_counts()
    train_users = list(user_train_cnt[user_train_cnt >= filter_train].index)
    user_test_cnt = receipts_test[["party_rk", "item"]].drop_duplicates().party_rk.value_counts()
    test_users = list(user_test_cnt[user_test_cnt >= filter_test].index)

    users_final = set(train_users).intersection(set(test_users))

    receipts_train = receipts_train[receipts_train.party_rk.isin(users_final)]
    receipts_test = receipts_test[receipts_test.party_rk.isin(users_final)]

    assert receipts_train.party_rk.nunique() == receipts_test.party_rk.nunique()

    return receipts_train, receipts_test, list(users_final)

In [ ]:
# Date for getting 75% train and 25% test (by time)
split_date_index = round(3 * receipts.ymd.nunique() / 4)
split_date = sorted(receipts.ymd.unique())[split_date_index]

receipts_train, receipts_test, users_final = split_by_time(receipts,
                                                           split_date=split_date,
                                                           filter_train=20, 
                                                           filter_test=20)

Перевод в sparse матрицы, поскольку матрицы крайне разреженные.

In [ ]:
# Encode party_rks and items 

user_encoder, item_encoder = LabelEncoder(), LabelEncoder()
user_encoder.fit(users_final)

all_items = set(receipts_test.item.unique()).union(set(receipts_train.item.unique()))
item_encoder.fit(list(all_items))

receipts_train['party_rk_id'] = user_encoder.transform(receipts_train['party_rk'])
receipts_train['item_type_id'] = item_encoder.transform(receipts_train['item'])

receipts_test['party_rk_id'] = user_encoder.transform(receipts_test['party_rk'])
receipts_test['item_type_id'] = item_encoder.transform(receipts_test['item'])

matrix_shape = len(user_encoder.classes_), len(item_encoder.classes_)

train_sparse = coo_matrix((list(receipts_train.item_cnt.astype(np.float32)), 
                           (list(receipts_train.party_rk_id.astype(np.int64)), 
                            list(receipts_train.item_type_id.astype(np.int64)))), shape=matrix_shape)


test_sparse = coo_matrix((list(receipts_test.item_cnt.astype(np.float32)),
                          (list(receipts_test.party_rk_id.astype(np.int64)), 
                           list(receipts_test.item_type_id.astype(np.int64)))), shape=matrix_shape)

In [ ]:
train_sparse.shape, test_sparse

Оценим LightFM

In [ ]:
# MNAP@20 for evaluating LightFM 

def metric_lightfm(model, test_sparse, user_features, indices, total):
    
    ranks = model.predict_rank(test_sparse, num_threads=60, check_intersections=True, \
                               user_features=user_features)
    mask = ranks.copy()
    mask.data = np.less(mask.data, 20, mask.data)
    ranks.data += 1
    ranks.data = ranks.data * mask.data
    ranks.eliminate_zeros()
    ranks = ranks.tolil().data
    average_precision_sum = 0.0
    for x in indices:
        n_correct_items = 0
        precision = 0
        for y in sorted(ranks[x]):
            n_correct_items += 1
            precision += n_correct_items / y
        average_precision_sum += precision / min(total[x], 20)
    average_precision_sum /= len(indices)
    return average_precision_sum

In [ ]:
model_lfm_full = LightFM(no_components=100, loss='warp', random_state=42, 
                    user_alpha=6e-5, item_alpha=2e-5, learning_rate=0.01, max_sampled=15)

total = test_sparse.getnnz(axis=1)
indices = np.nonzero(total)[0]

maps = []
epochs = 3
for rounds in tqdm(range(7)):
    %time model_lfm_full.fit_partial(train_sparse, sample_weight=train_sparse, epochs=epochs, \
                                num_threads=40, user_features=train_sparse)
    curr_metric = metric_lightfm(model_lfm_full, test_sparse, test_sparse, indices, total)
    maps.append(curr_metric)
    print(curr_metric)

BM25 Recommender

In [ ]:
from implicit.nearest_neighbours import BM25Recommender
from implicit.evaluation import mean_average_precision_at_k

In [ ]:
model_bm25 = BM25Recommender(K=150, K1=0.2, B=1.)
model_bm25.fit(train_sparse.T.tocsr())
print('BM25 MAP@20 ', mean_average_precision_at_k(model_bm25, train_sparse, test_sparse, K=20, num_threads=80))

Посчитаем предсказания для второго уровня модели.

In [ ]:
def bm25_data(model_bm25, train_sparse, test_sparse, top=50):
    
    bm25_dict, bm25_pairs = dict(), list()
    users_test = sorted(list(set(coo_matrix(test_sparse).row)))
    train_sparse = csr_matrix(train_sparse)
    for i in tqdm(range(test_sparse.shape[0])):
        rec_list, rec_set = [], set()
        recommendations = model_bm25.recommend(i, train_sparse, N=1000, filter_already_liked_items=False)
        for rank, recom in enumerate(recommendations):
            if recom[1] > 0:
                bm25_dict[(i, recom[0])] = (recom[1], rank + 1)
                if len(rec_list) >= top:
                    break
                elif len(rec_list) < top:
                    rec_list.append((i, recom[0]))
                    rec_set.add(recom[0])
        bm25_pairs.extend(rec_list)
        
    return bm25_dict, bm25_pairs


def lightfm_data(model_lfm, users_test, items_test, top=50):
    
    lightfm_dict, lightfm_pairs = dict(), list()
    user_biases, item_biases = model_lfm.user_biases[users_test], model_lfm.item_biases[items_test]
    item_emb = model_lfm.item_embeddings[items_test]
    user_emb = model_lfm.user_embeddings[users_test]
    
    preds = user_emb.dot(item_emb.T) + user_biases.reshape(-1,1) + item_biases.reshape(1,-1)
    preds_items = (-preds).argsort(axis=1)
    preds_scores = -np.sort(-preds, axis=1)
    items_lfm = dict(list(zip(users_test, preds_items)))
    lfm_scores = dict(list(zip(users_test, preds_scores)))
    
    user_biases_series = pd.Series(user_biases, index=users_test)
    item_biases_series = pd.Series(item_biases, index=items_test)

    for ids, user in tqdm(enumerate(users_test)):
        current_extend = list()
        current_scores = lfm_scores[user]
        
        for rank, (item, value) in enumerate(zip(items_lfm[user], current_scores)):
            if len(current_extend) >= top:
                break
            elif len(current_extend) < top:
                lightfm_dict[(user, item)] =  (value, rank + 1)
                current_extend.append((user, item))                
        lightfm_pairs.extend(current_extend)
        
    user_emb = pd.DataFrame(model_lfm.user_embeddings[users_test], index=users_test)
    
    return lightfm_pairs, lightfm_dict, user_biases_series, item_biases_series, user_emb

In [ ]:
bm25_dict, bm25_pairs = bm25_data(model_bm25, train_sparse, test_sparse, top=50)

users_test = sorted(list(set(coo_matrix(train_sparse).row)))
items_test = sorted(list(set(train_sparse.col)))

lightfm_pairs, lightfm_dict, user_biases_series, item_biases_series, \
user_emb = lightfm_data(model_lfm_full, users_test, items_test, top=50)

total_pairs = list(set(bm25_pairs).union(set(lightfm_pairs)))
data_all_pairs = [pair + bm25_dict.get(pair, (np.nan, np.nan) + lightfm_dict.get(pair, (np.nan, np.nan))) for pair in tqdm(total_pairs)]
data_all_pairs_df = pd.DataFrame(data_all_pairs, columns=["client_id", "item_id", "bm25_score", "bm25_rank", "lfm_score", "lfm_rank"])

In [ ]:
# Purchases in test period for making targets

purchases = list()

for k in tqdm(range(test_sparse.shape[0])):
    test_sparse = csr_matrix(test_sparse)
    cx = scipy.sparse.coo_matrix(test_sparse[k])
    purchased_items, client_id = [], []
    client_id.append(k)
    for i,j,v in zip(cx.row, cx.col, cx.data):
        purchased_items.append(j)
    for i in list(itertools.product(client_id, purchased_items)):
        purchases.append(i)
        
def change_dtype(df):
    df.client_id = df.client_id.astype(np.int32)
    df.item_id = df.item_id.astype(np.int16)
    df.bm25_score = df.bm25_score.astype(np.float32)
    df.bm25_rank = df.bm25_rank.astype(np.float16)
    df.lfm_score = df.lfm_score.astype(np.float32)
    df.lfm_rank = df.lfm_rank.astype(np.float16)
    return df

def purchases2dict(purchases):
    data_true = {}
    for i in tqdm(purchases):
        curr, item = i[0], int(i[1])
        if curr not in data_true:
            data_true[curr] = list()
            data_true[curr].append(item)
        else:
            data_true[curr].append(item)
    for i in tqdm(data_true.keys()):
        data_true[i] = set(data_true[i])
    return data_true

data_all_pairs_df = change_dtype(data_all_pairs_df)
data_true = purchases2dict(purchases)

In [ ]:
predictions = data_all_pairs_df.copy()
predictions.head()

In [ ]:
%load_ext Cython

In [ ]:
%%cython

def average_precision(
        dict data_true,
        dict data_predicted,
        const unsigned long int k
) -> float:
    cdef:
        unsigned long int n_items_predicted
        unsigned long int n_items_true
        unsigned long int n_correct_items
        unsigned long int item_idx

        double average_precision_sum
        double precision

        set items_true
        list items_predicted

    if not data_true:
        raise ValueError('data_true is empty')

    average_precision_sum = 0.0

    for key, items_true in data_true.items():
        items_predicted = data_predicted.get(key, [])

        n_items_true = len(items_true)
        n_items_predicted = min(len(items_predicted), k)

        if n_items_true == 0 or n_items_predicted == 0:
            continue

        n_correct_items = 0
        precision = 0.0

        for item_idx in range(n_items_predicted):
            if items_predicted[item_idx] in items_true:
                n_correct_items += 1
                precision += <double>n_correct_items / <double>(item_idx + 1)

        average_precision_sum += <double>precision / <double>min(n_items_true, k)

    return average_precision_sum / <double>len(data_true)


def metric(true_data, predicted_data, k=20):
    true_data_set = {k: set(v) for k, v in true_data.items()}

    return average_precision(true_data_set, predicted_data, k=k)

In [ ]:
def fill_nan(data):
    data.bm25_score.fillna(random.uniform(0, 1), inplace=True)
    data.bm25_rank.fillna(random.randint(20, 40), inplace=True)
    data.lfm_score.fillna(random.uniform(0, 1), inplace=True)
    data.lfm_rank.fillna(random.randint(20, 40), inplace=True)
    return data

predictions = fill_nan(predictions)

In [ ]:
items_dict = dict(zip(receipts_train.item_type_id, receipts_train.item))
users_dict = dict(zip(receipts_train.party_rk_id, receipts_train.party_rk))

predictions['party_rk'] = predictions['client_id'].map(users_dict)
predictions['item'] = predictions['item_id'].map(items_dict)

receipts_test['target'] = 1

predictions_labeled = pd.merge(predictions, receipts_test, how='left', left_on=['client_id', 'item_id'],
                              right_on=['party_rk_id', 'item_type_id'])
predictions_labeled.drop(columns=['party_rk_y', 'item_cnt', 'item_y', 'party_rk_id'], inplace=True)
predictions_labeled['target'] = predictions_labeled['target'].fillna(0)

data = predictions_labeled[['party_rk_x', 'item_id', 'bm25_score', 'bm25_rank', 'lfm_score', 
                                          'lfm_rank', 'target']]

In [ ]:
sns.heatmap(data.corr(), annot=True);

In [ ]:
users_ids = data.party_rk_x.unique()
index_train, index_test = train_test_split(users_ids, train_size=0.7, random_state=42)

X_train = data[data.party_rk_x.isin(index_train)].reset_index(drop=True)
X_test = data[data.party_rk_x.isin(index_test)].reset_index(drop=True)

X_train[['party_rk_x', 'item_id']].drop_duplicates(inplace=True)
X_test[['party_rk_x', 'item_id']].drop_duplicates(inplace=True)

X_train.set_index(['party_rk_x', 'item_id'], inplace=True)
X_test.set_index(['party_rk_x', 'item_id'], inplace=True)

y_train, y_test = X_train.pop("target"), X_test.pop("target")

train_data, test_data = lightgbm.Dataset(X_train, y_train), lightgbm.Dataset(X_test, y_test)

In [ ]:
params = {'application': 'binary', 'objective': 'binary',
    'metric': 'binary_logloss', "num_threads": 20, 'verbose': 1, 'learning_rate': 0.0001}

model = lightgbm.train(params, train_data, valid_sets=test_data, num_boost_round=800,
                       early_stopping_rounds=20, verbose_eval=10)

# model = lightgbm.train(params, train_data, num_boost_round=2000)

In [ ]:
lgb_test = data.copy()
lgb_test[['party_rk_x', 'item_id']].drop_duplicates(inplace=True)
lgb_test.set_index(['party_rk_x', 'item_id'], inplace=True)
lgb_test.drop(columns='target', inplace=True)
lgb_test["lgb_score"] = model.predict(lgb_test, num_iteration=model.best_iteration)
lgb_test = lgb_test.set_index('lgb_score', append=True).sort_values("lgb_score", ascending=False)
lgb_test.drop_duplicates(inplace=True)

data_predicted = dict()
lgb_test.reset_index(inplace=True)
for user, group in tqdm(lgb_test.groupby("party_rk_x")):
    data_predicted[user] = list(group.item_id)[:20]

In [ ]:
data_predicted_unique = {k:list(set(v)) for (k,v) in data_predicted.items()}

data_predicted_recoms = {}
for k, v in tqdm(data_predicted_unique.items()):
    recom = []
    for j in v:
        try:
            recom.append(items_dict[j])
        except:
            recom.append(np.nan)
    data_predicted_recoms[k] = recom

data_predicted_df = pd.DataFrame.from_dict(data_predicted_recoms).T


data_true = {}
lgb_test = X_test.copy()
lgb_test['target'] = y_test
data_true_df = lgb_test[lgb_test['target'] != 0].reset_index()
for user, group in tqdm(data_true_df.groupby("party_rk_x")):
    data_true[user] = list(group.item_id)
    
intersect = set(data_true.keys()) & set(data_predicted.keys()) # by users
data_true_inter = {k:v for (k,v) in data_true.items() if k in intersect}
data_predicted_inter = {k:v for (k,v) in data_predicted.items() if k in intersect}

for top in [1, 5, 10, 20]:
    print(f'mnap@{top}=', metric(data_true_inter, data_predicted_inter, k=top))

<img src='https://raw.githubusercontent.com/anamarina/RecSys_course/main/week3/images/validation.png' width=500 height=300>